In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/linking-writing-processes-to-writing-quality/sample_submission.csv
/kaggle/input/linking-writing-processes-to-writing-quality/test_logs.csv
/kaggle/input/linking-writing-processes-to-writing-quality/train_scores.csv
/kaggle/input/linking-writing-processes-to-writing-quality/train_logs.csv


## 1. EDA

In [2]:
# Reading the files
train_df = pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/train_logs.csv')
test_df = pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/test_logs.csv')
train_scores = pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/train_scores.csv')

In [3]:
# checking train_df and train_scores
#train_df

In [4]:
#train_scores

In [5]:
# Checking the column list
#train_df.columns

In [6]:
# Checking data types of each column
#train_df.dtypes

In [7]:
# checking for null values
#train_df.isnull().sum()

In [8]:
# segregrating categoricl and numerical column
categorical_columns = []
numerical_columns = []
for i in train_df.columns:
    if train_df[i].dtype == 'O':
        categorical_columns.append(i)
    else:
        numerical_columns.append(i)

In [9]:
categorical_columns.remove('id')
numerical_columns.remove('event_id')


In [10]:
# print(categorical_columns)
# print(numerical_columns)

In [11]:
# for i in categorical_columns:
#     print (i)
#     print(train_df[i].unique())
#     print("--------------------------------------------")

In [12]:
# Creating categorical features at id level for all the categorical columns
# Below are helper functions that will be utilized to create the categorical features
# returns a dataframe with index and all the unique value of categorical columns as columns as defined 
# in below list

from collections import Counter

activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste','Move from','Other']
events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',', 'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Other']
text_changes = ['q', ' ', 'NoChange', '.', ',', '\n', "'", '"', '-', '?', ';', '=', '/', '\\', ':','Other']
    

def activity_counts(df,col):
    # Agrregrating column on 'Move from'
    df[col]  = df[col].apply(lambda x : 'Move From' if 'Move From' in x else x)
    temp_df = df.groupby('id').agg({col: list}).reset_index()
    final_count = []
    for rows,ids in list(zip(temp_df[col].values,temp_df['id'].values)):
        items = list(Counter(rows).items())
        di = {}
        di['id'] = ids
        for act in activities:
            di[act] = 0
        for act,count in items:
            if act in di:
                di[act] = count
            else:
                di['Other'] += count
        final_count.append(di)
    final_count = pd.DataFrame(final_count)
    new_cols = [f'{col}_{i}' if i !='id' else i for i in final_count.columns ]
    final_count.columns = new_cols

    return final_count
        
# Same function for down and up
def event_counts(df,col):
    temp_df = df.groupby('id').agg({col: list}).reset_index()
    final_count = []
    for rows,ids in list(zip(temp_df[col].values,temp_df['id'].values)):
        items = list(Counter(rows).items())
        di = {}
        di['id'] = ids
        for act in events:
            di[act] = 0
        for act,count in items:
            if act in di:
                di[act] = count
            else:
                di['Other'] += count
        final_count.append(di)
    final_count = pd.DataFrame(final_count)
    new_cols = [f'{col}_{l}' if i !='id' else i for l,i in enumerate(final_count.columns) ]
    final_count.columns = new_cols

    return final_count

def text_change(df,col):
    temp_df = df.groupby('id').agg({col: list}).reset_index()
    final_count = []
    for rows,ids in list(zip(temp_df[col].values,temp_df['id'].values)):
        items = list(Counter(rows).items())
        di = {}
        di['id'] = ids
        for act in text_changes:
            di[act] = 0
        for act,count in items:
            if act in di:
                di[act] = count
            else:
                di['Other'] += count
        final_count.append(di)
    final_count = pd.DataFrame(final_count)
    new_cols = [f'{col}_{l}' if i !='id' else i for l,i in enumerate(final_count.columns) ]
    final_count.columns = new_cols

    return final_count

In [13]:
def pre_process(df):
    
    data = pd.DataFrame()
    df["total_event_time"] = df["up_time"]-df["down_time"]
    data['events'] = df.groupby('id')['event_id'].max()
    data['up_time_sum'] = df.groupby('id')['up_time'].sum()
    data['up_time_mean'] = df.groupby('id')['up_time'].mean()
    data['up_time_std'] = df.groupby('id')['up_time'].std()

    data['down_time_sum'] = df.groupby('id')['down_time'].sum()
    data['down_time_mean'] = df.groupby('id')['down_time'].mean()
    data['down_time_std'] = df.groupby('id')['down_time'].std()
    
    data['event_time_sum'] = df.groupby('id')['total_event_time'].sum()
    data['event_time_mean'] = df.groupby('id')['total_event_time'].mean()
    data['event_time_std'] = df.groupby('id')['total_event_time'].std()

    data['action_time_sum'] = df.groupby('id')['action_time'].sum()
    data['action_time_max'] = df.groupby('id')['action_time'].max()
    data['action_time_mean'] = df.groupby('id')['action_time'].mean()
    data['action_time_std'] = df.groupby('id')['action_time'].std()

    data['down_up_ratio'] = df.groupby('id')['down_time'].mean() / df.groupby('id')['up_time'].mean()
    data['down_action_ratio'] = df.groupby('id')['down_time'].mean() / df.groupby('id')['action_time'].mean()
    data['up_action_ratio'] = df.groupby('id')['up_time'].mean() / df.groupby('id')['action_time'].mean()

    data['down_event_nunique'] = df.groupby('id')['down_event'].nunique()
    data['up_event_nunique'] = df.groupby('id')['up_event'].nunique()
    
    data['cursor_position_nunique'] = df.groupby('id')['cursor_position'].nunique()
    data['cursor_position_median'] = df.groupby('id')['cursor_position'].median()
    data['cursor_position_max'] = df.groupby('id')['cursor_position'].max()

    data['word_count_max'] = df.groupby('id')['word_count'].max()
    data['word_count_mean'] = df.groupby('id')['word_count'].mean()
    
    data = data.reset_index()
    temp1 = activity_counts(df,'activity')
    
    return_df = pd.merge(data,temp1,how='inner',on='id')
    del temp1
    
    
    temp2 = event_counts(df,'down_event')
    return_df = pd.merge(return_df,temp2,how='inner',on='id')
    del temp2

    temp3 = event_counts(df,'up_event')
    return_df = pd.merge(return_df,temp3,how='inner',on='id')
    del temp3

    temp4 = text_change(df,'text_change')
    return_df = pd.merge(return_df,temp4,how='inner',on='id')
    del temp4

    
    return_df.set_index('id',inplace=True)
    
    return return_df

In [14]:
X_train = pre_process(train_df)
Y_train = train_scores.set_index('id')
df_train = X_train.join(Y_train)

In [15]:
#X_train

## FITTING THE MODEL

In [16]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test =train_test_split(df_train.iloc[:,:-1],df_train.iloc[:,-1],test_size = 0.3)

In [17]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import lightgbm as lgb
params_lgbm = {
        'n_estimators': 1140, 'max_depth': 3, 'learning_rate': 0.012192985298639868,
               'min_child_weight': 2.5028737839599557, 'min_child_samples': 40,
               'subsample': 0.5833615591152317, 'subsample_freq': 1, 'colsample_bytree': 0.734238785736391
    }
lgbm_opt = lgb.LGBMRegressor(**params_lgbm)
lgbm_opt.fit(X_train,y_train,eval_set=[(X_test,y_test)],early_stopping_rounds=100,verbose=False)
preds  = lgbm_opt.predict(X_test)


/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


In [18]:
print("MAE:",mean_absolute_error(y_test,preds))
print("MSE:",mean_squared_error(y_test,preds))
print("RMSE:",mean_squared_error(y_test,preds,squared=False))

MAE: 0.5072188780404872
MSE: 0.43400751317478087
RMSE: 0.6587924659365655


In [19]:
from xgboost import XGBRegressor
params_xgb = {'n_estimators': 786, 'max_depth': 7, 'min_child_weight': 2.039057805276082,
              'learning_rate': 0.014416140789003485, 'subsample': 0.4709832796447838,
              'gamma': 0.944842834910575, 'colsample_bytree': 0.3665662093440971,
              'colsample_bylevel': 0.5005441850291041, 'colsample_bynode': 0.5984372995917981}

xgb_opt = XGBRegressor(**params_xgb)#tree_method = "gpu_hist")
xgb_opt.fit(X_train,y_train)
preds = xgb_opt.predict(X_test)

In [20]:
print("MAE:",mean_absolute_error(y_test,preds))
print("MSE:",mean_squared_error(y_test,preds))
print("RMSE:",mean_squared_error(y_test,preds,squared=False))

MAE: 0.5046544463808003
MSE: 0.4381614474256955
RMSE: 0.6619376461765077


In [21]:
from sklearn.ensemble import VotingRegressor

vr_opt = VotingRegressor([("xgb",xgb_opt),("lgbm",lgbm_opt)],weights=[0.3,0.7])
vr_opt.fit(X_train,y_train)

preds = vr_opt.predict(X_test)

In [22]:
print("MAE:",mean_absolute_error(y_test,preds))
print("MSE:",mean_squared_error(y_test,preds))
print("RMSE:",mean_squared_error(y_test,preds,squared=False))

MAE: 0.5059988758709525
MSE: 0.43514492181233155
RMSE: 0.6596551537070953


## SUBMISSION

In [23]:
sub_df = pre_process(test_df)
sub_predict = vr_opt.predict(sub_df)
sub_dict = {'id':sub_df.index, 'score':sub_predict}
sub_df = pd.DataFrame(sub_dict)
sub_df.to_csv("submission.csv",index=False)